In [ ]:
import random
import numpy as np
from torchvision import datasets, transforms

In [ ]:
# Let's read the mnist dataset

def load_mnist(path='.'):
    !wget www.di.ens.fr/~lelarge/MNIST.tar.gz
    !tar -zxvf MNIST.tar.gz

    from torchvision.datasets import MNIST

    train_set = MNIST(root = './', train=True, download=True)
    test_set = MNIST(root = './', train=False, download=True)

    x_train = train_set.data.numpy()
    _y_train = train_set.targets.numpy()
    
    x_test = test_set.data.numpy()
    _y_test = test_set.targets.numpy()
    
    x_train = x_train / 255.
    x_test = x_test / 255.

    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1
    
    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_mnist()

--2021-03-30 21:29:56--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-03-30 21:29:57--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [         <=>        ]  33.20M  6.52MB/s    in 6.1s    

2021-03-30 21:30:04 (5.45 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

In this exercise your task is to fill in the gaps in this code by implementing the backpropagation algorithm
Once this is done, you can run the network on the MNIST example and see how it performs. Feel free to play with the parameters.

If you found this task too easy, try to implement a "fully vectorized" version, i.e. one using matrix operations instead of going over examples one by one.

In [ ]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z)*(1-sigmoid(z))

class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
    def feedforward(self, a):
        # Run the network on a single case
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    def update_mini_batch(self, x_mini_batch, y_mini_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in zip(x_mini_batch, y_mini_batch):
            delta_nabla_b, delta_nabla_w = self.backprop(x.reshape(784,1), y.reshape(10,1))
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(x_mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(x_mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
        # print(self.weights)
        
    def backprop(self, x, y):
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.
        
        # First initialize the list of gradient arrays
        delta_nabla_b = []
        delta_nabla_w = []
        
        # Then go forward remembering all values before and after activations
        # in two other array lists

        gs = [x]
        fs = []
        for b, w in zip(self.biases, self.weights):
            fs.append(np.dot(w, gs[-1]) + b)
            gs.append(sigmoid(fs[-1]))

        # Now go backward from the final cost applying backpropagation
        
        dg = gs[-1] - y
        dfs = []
        for w, g, f in reversed(list(zip(self.weights, gs[1:], fs))):
            dfs.append(np.multiply(dg, sigmoid_prime(f)))
            dg = np.matmul(w.T, dfs[-1])
        
        for df, g in zip(reversed(dfs), gs[:-1]):
          delta_nabla_w.append(np.matmul(df, g.T))
          delta_nabla_b.append(np.sum(df, axis=1)[:, np.newaxis])
        
        return delta_nabla_b, delta_nabla_w

    def evaluate(self, x_test_data, y_test_data):
        # Count the number of correct answers for test_data
        test_results = [(np.argmax(self.feedforward(x_test_data[i].reshape(784,1))), np.argmax(y_test_data[i]))
                        for i in range(len(x_test_data))]
        # return accuracy
        return np.mean([int(x == y) for (x, y) in test_results])
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
            # for i in range(1):
                x_mini_batch = x_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)] 
                y_mini_batch = y_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)] 
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate(x_test, y_test)))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=50, mini_batch_size=100, eta=3., test_data=(x_test, y_test))



Epoch: 0, Accuracy: 0.7468
Epoch: 1, Accuracy: 0.8364
Epoch: 2, Accuracy: 0.8763
Epoch: 3, Accuracy: 0.8898
Epoch: 4, Accuracy: 0.8981
Epoch: 5, Accuracy: 0.9036
Epoch: 6, Accuracy: 0.909
Epoch: 7, Accuracy: 0.9116
Epoch: 8, Accuracy: 0.9139
Epoch: 9, Accuracy: 0.9168
Epoch: 10, Accuracy: 0.9188
Epoch: 11, Accuracy: 0.9205
Epoch: 12, Accuracy: 0.9219
Epoch: 13, Accuracy: 0.9238
Epoch: 14, Accuracy: 0.925
Epoch: 15, Accuracy: 0.9256
Epoch: 16, Accuracy: 0.9266
Epoch: 17, Accuracy: 0.9282
Epoch: 18, Accuracy: 0.9297
Epoch: 19, Accuracy: 0.9304
Epoch: 20, Accuracy: 0.9316
Epoch: 21, Accuracy: 0.9326
Epoch: 22, Accuracy: 0.9332
Epoch: 23, Accuracy: 0.9341
Epoch: 24, Accuracy: 0.9347
Epoch: 25, Accuracy: 0.9352
Epoch: 26, Accuracy: 0.9355
Epoch: 27, Accuracy: 0.9363
Epoch: 28, Accuracy: 0.9365
Epoch: 29, Accuracy: 0.9365
Epoch: 30, Accuracy: 0.9366
Epoch: 31, Accuracy: 0.9369
Epoch: 32, Accuracy: 0.9373
Epoch: 33, Accuracy: 0.9381
Epoch: 34, Accuracy: 0.9388
Epoch: 35, Accuracy: 0.9396
Epoc